In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-pubmed-ids/train_pubmed_ids_all.csv
/kaggle/input/abstract-embeds-final/tfidf_go_sparse_matrix_test.npz
/kaggle/input/abstract-embeds-final/protein_ids_train_sparse.csv
/kaggle/input/abstract-embeds-final/Go_vocab_terms.csv
/kaggle/input/abstract-embeds-final/protein_ids_test_sparse.csv
/kaggle/input/abstract-embeds-final/tfidf_go_sparse_matrix_train.npz
/kaggle/input/train-labels-sorted-zong/Ankh_train_labels_sorted.npy
/kaggle/input/preprocessed-data-testdi/preprocessed_data_test.csv
/kaggle/input/proprocessed-abstract-train/preprocessed_data.csv
/kaggle/input/test-ids-sorted-zong/t5_labels_sorted.npy
/kaggle/input/test-pubmed-ids/test_pubmed_ids_all.xlsx
/kaggle/input/cafa-5-protein-function-prediction/sample_submission.tsv
/kaggle/input/cafa-5-protein-function-prediction/IA.txt
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset-taxon-list.tsv
/kaggle/inpu

In [2]:
!pip install obonet

In [3]:
train_preprocessed = pd.read_csv('/kaggle/input/proprocessed-abstract-train/preprocessed_data.csv')

In [4]:
test_preprocessed = pd.read_csv('/kaggle/input/preprocessed-data-testdi/preprocessed_data_test.csv')

In [5]:
train_preprocessed

,Protein ID,Preprocessed Abstract
0,Q76MV3,The Cox17p rodent cells. Previous works isolat...
1,Q8I2B1,Genome malaria parasite Plasmodium falciparum....
2,Q95S63,Effect chloroquine cultured fibroblasts: lysos...
3,Q9VHH8,The Drosophila melanogaster. The fly Drosophil...
4,C8Z5H8,Eukaryote-to-eukaryote transfer wine Saccharom...
...,...,...
134666,O51056,"Genomic Lyme spirochaete, Borrelia burgdorferi..."
134667,Q8C6P8,A zinc fingers down-regulated differentiation....
134668,P10984,Wild-type Caenorhabditis elegans. We Caenorhab...
134669,P07839,A cDNA Chlamydomonas reinhardtii preferredoxin...


In [6]:
test_preprocessed

,Protein ID,Preprocessed Abstract
0,Q9CQV8,Formate assay body fluids: application methano...
1,P62259,"The 14-3-3 epsilon isoform, regulator whose pa..."
2,P68510,The Cdk-like PCTAIRE-1 associates p11 14-3-3 p...
3,P61982,Formate assay body fluids: application methano...
4,O70456,Formate assay body fluids: application methano...
...,...,...
141814,P08380,"The toxin RpIII sea anemone, Radianthus paumot..."
141815,C0HK72,Peptide fingerprinting sea anemone Heteractis ...
141816,C0HK73,Peptide fingerprinting sea anemone Heteractis ...
141817,C0HK74,Peptide fingerprinting sea anemone Heteractis ...


In [7]:
import obonet
import re
import nltk
from nltk.corpus import stopwords
import string
# Specify the path to your OBO file
obo_file_path = '/kaggle/input/cafa-5-protein-function-prediction/Train/go-basic.obo'

# Read the OBO file
graph = obonet.read_obo(obo_file_path)

terms = [data.get('name') for _, data in graph.nodes.items()]

# Combine all terms into a single string
all_terms_text = ' '.join(terms)

# Clean the text
cleaned_text = re.sub(r'\W', ' ', all_terms_text.lower())  # Remove non-alphanumeric characters
cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # Remove extra whitespaces

# Tokenize the text
tokens = nltk.word_tokenize(cleaned_text)

# Remove stopwords
stopwords = set(stopwords.words('english'))
filtered_words = [word for word in tokens if word.lower() not in stopwords and word not in string.punctuation and not word.isdigit() and len(word)>2]

# Remove duplicate words
filtered_words_all = list(set(filtered_words))

#print(filtered_words)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
df_go_vocab = pd.DataFrame(filtered_words_all, columns=["vocab"])
df_go_vocab.to_csv('/kaggle/working/Go_vocab_terms.csv', index=False)

In [9]:
df_go_vocab

,vocab
0,methylselenocysteine
1,ripoptosome
2,erf1
3,hydroxymethylcephem
4,glucosylmutase
...,...
10274,alanylation
10275,methyloct
10276,maltotetraohydrolase
10277,vitexifolin


In [10]:
set_a = set(test_preprocessed['Protein ID'])
set_b = set(train_preprocessed['Protein ID'])

# Find elements in set_a but not in set_b
diff = set_a - set_b

# Convert the result back to a list (if desired)
diff_list = list(diff)

# Print the result
print(len(diff_list))

68176


In [11]:
train_test_preprocessed = pd.concat([train_preprocessed, test_preprocessed], axis=0, ignore_index=True)
print(train_test_preprocessed)

        Protein ID                              Preprocessed Abstract
0           Q76MV3  The Cox17p rodent cells. Previous works isolat...
1           Q8I2B1  Genome malaria parasite Plasmodium falciparum....
2           Q95S63  Effect chloroquine cultured fibroblasts: lysos...
3           Q9VHH8  The Drosophila melanogaster. The fly Drosophil...
4           C8Z5H8  Eukaryote-to-eukaryote transfer wine Saccharom...
...            ...                                                ...
276485      P08380  The toxin RpIII sea anemone, Radianthus paumot...
276486      C0HK72  Peptide fingerprinting sea anemone Heteractis ...
276487      C0HK73  Peptide fingerprinting sea anemone Heteractis ...
276488      C0HK74  Peptide fingerprinting sea anemone Heteractis ...
276489  A0A3G2FQK2  Atomic polypeptide backbones myohemerythrin he...

[276490 rows x 2 columns]


In [12]:
train_test_preprocessed.drop_duplicates(subset=['Protein ID'], inplace=True)


In [13]:
train_test_preprocessed

,Protein ID,Preprocessed Abstract
0,Q76MV3,The Cox17p rodent cells. Previous works isolat...
1,Q8I2B1,Genome malaria parasite Plasmodium falciparum....
2,Q95S63,Effect chloroquine cultured fibroblasts: lysos...
3,Q9VHH8,The Drosophila melanogaster. The fly Drosophil...
4,C8Z5H8,Eukaryote-to-eukaryote transfer wine Saccharom...
...,...,...
276485,P08380,"The toxin RpIII sea anemone, Radianthus paumot..."
276486,C0HK72,Peptide fingerprinting sea anemone Heteractis ...
276487,C0HK73,Peptide fingerprinting sea anemone Heteractis ...
276488,C0HK74,Peptide fingerprinting sea anemone Heteractis ...


In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import save_npz
abstracts_df = train_test_preprocessed
print('creating_tfidf')

# Create the TfidfVectorizer
vectorizer = TfidfVectorizer(vocabulary=filtered_words_all)
vectorizer.fit(abstracts_df['Preprocessed Abstract'])


creating_tfidf


TfidfVectorizer(vocabulary=['methylselenocysteine', 'ripoptosome', 'erf1',
                            'hydroxymethylcephem', 'glucosylmutase',
                            'hydroxypalmitate', 'mouth', 'perillyl',
                            'threonylcarbomyladenosine', 'proline',
                            'deoxycorticosterone', 'isobazzanene', 'sperm',
                            'lys63', 'oncosis', 'contact', 'arachidoyl',
                            'lathosterol', 'pyrimidinone', 'chromo',
                            'butenoate', 'rad51b', 'generated', 'cutinase',
                            'uag', 'decylhomocitrate', 'endocrocin',
                            'pantothenase', 'carboxyvinyltransferase',
                            'intronic', ...])

In [15]:
tfidf_train_vectors = vectorizer.transform(train_preprocessed['Preprocessed Abstract'])
tfidf_test_vectors = vectorizer.transform(test_preprocessed['Preprocessed Abstract'])


# Convert the TF-IDF vectors to an array
train_vectors = tfidf_train_vectors.toarray()
test_vectors = tfidf_test_vectors.toarray()

# Update the abstracts dataframe with the embedding vectors
train_preprocessed['Embedding Vector'] = list(train_vectors)
test_preprocessed['Embedding Vector'] = list(test_vectors)




In [16]:
# Save the TF-IDF sparse matrices
save_npz('/kaggle/working/tfidf_go_sparse_matrix_train.npz', tfidf_train_vectors)
save_npz('/kaggle/working/tfidf_go_sparse_matrix_test.npz', tfidf_test_vectors)

# Save the protein IDs for both dataframes
train_preprocessed['Protein ID'].to_csv('/kaggle/working/protein_ids_train.csv', index=False)
test_preprocessed['Protein ID'].to_csv('/kaggle/working/protein_ids_test.csv', index=False)


In [17]:
train_ids_np = np.load('/kaggle/input/train-labels-sorted-zong/Ankh_train_labels_sorted.npy')

In [18]:
train_ids_df = pd.DataFrame(train_ids_np, columns = ['ID'])

In [19]:
train_ids_df


,ID
0,A0A009IHW8
1,A0A021WW32
2,A0A021WZA4
3,A0A023FBW4
4,A0A023FBW7
...,...
142241,X6RKS3
142242,X6RLN4
142243,X6RLP6
142244,X6RLR1


In [20]:
embed_ids = pd.read_csv('/kaggle/input/abstract-embeds-final/protein_ids_train_sparse.csv')

In [21]:
embed_ids

,Protein ID
0,Q76MV3
1,Q8I2B1
2,Q95S63
3,Q9VHH8
4,C8Z5H8
...,...
134666,O51056
134667,Q8C6P8
134668,P10984
134669,P07839


In [22]:
from scipy.sparse import load_npz
sparse_matrix_train = load_npz('/kaggle/input/abstract-embeds-final/tfidf_go_sparse_matrix_train.npz')

df_npz = pd.DataFrame.sparse.from_spmatrix(sparse_matrix_train)
df_npz['Protein_ID'] = embed_ids['Protein ID'].values

# Convert series_train_protein_ids to DataFrame
df_ids = train_ids_df

# Reset the index of df_npz to allow the merge
#df_npz_reset = df_npz.reset_index()

# Perform the merge with a left join
df_train = pd.merge(df_ids, df_npz, how='left', left_on='ID', right_on = 'Protein_ID')

# Fill NaN values with zeros
df_train[df_npz.columns] = df_train[df_npz.columns].fillna(0)

# Set 'ID' as the index of the DataFrame again
df_train.set_index('Protein_ID', inplace=True)
df_train = df_train.drop('ID', axis=1)


In [23]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,10269,10270,10271,10272,10273,10274,10275,10276,10277,10278
Protein_ID,,,,,,,,,,,,,,,,,,,,,
A0A009IHW8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
A0A021WW32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
A0A021WZA4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013019,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009814,0.0
A0A023FBW4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
A0A023FBW7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X6RKS3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
X6RLN4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.030409,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
X6RLP6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [24]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,10269,10270,10271,10272,10273,10274,10275,10276,10277,10278
Protein_ID,,,,,,,,,,,,,,,,,,,,,
A0A009IHW8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
A0A021WW32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
A0A021WZA4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013019,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009814,0.0
A0A023FBW4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
A0A023FBW7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X6RKS3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
X6RLN4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.030409,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
X6RLP6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [25]:

# Get the feature names (words) from the original TfidfVectorizer
feature_names = vectorizer.get_feature_names_out()

# Select the row from df_train that corresponds to the given Protein ID
protein_id = 'A0A009IHW8'
row = df_train.loc[protein_id]

# Get the TF-IDF values for that row
tfidf_values = row[:-1]  # Assuming the last column is 'Protein_ID'

# Extract the words that have non-zero TF-IDF values
non_zero_indices = tfidf_values[tfidf_values != 0].index
non_zero_words = [feature_names[i] for i in non_zero_indices]

print(f"Words with non-zero TF-IDF values for Protein ID '{protein_id}': {non_zero_words}")

Words with non-zero TF-IDF values for Protein ID 'A0A009IHW8': ['fucokinase', 'nephron', 'apoptosis', 'microtubule', 'dependent', 'homomeric', 'phenylpyruvate', 'aminotransferase', 'prolipoprotein', 'methylcyclohexa', '3beta', 'imp', 'leptomycin', 'detoxification', 'hydroxymethylsilanetriol', 'm5u54', 'glycosylated', 'irhom2', 'columnal', 'adult', 'mannosylphosphodecaprenol', 'triggering', 'day', 'passenger', 'mhf', 'h3t3', 'macromolecule', 'glycerol', 'rbl', 'endo', 'acylsphingosine', 'anesthesia', 'albumen', 'fhl2', 'fabricius', 'symmetrical', 'enterocyte', 'glucuronoarabinoxylan', 'deoxygalactonokinase', 'theca', 'dealkylase', 'borate', 'geranylfarnesyl', 'germacrene', 'cysteate', 'brassinolide', 'auxin', 'ectoderm', 'cortisone', 'cd4', 'rave', 'endowment', 'guanidinobutyraldehyde', 'methylphosphothioglycerate', 'endocardium', 'nicotianamine', 'ccr4', 'socket', 'feminization', 'propan', 'r26', 'ectexine', 'methylnorlaudanosoline', 'palmitate', 'train', 'acetylmuramoyl', 'intergenic'

In [26]:
def get_non_zero_words(protein_id, df, vectorizer):
    # Get the feature names (words) from the original TfidfVectorizer
    feature_names = vectorizer.get_feature_names_out()

    # Select the row from df that corresponds to the given Protein ID
    row = df.loc[protein_id]

    # Get the TF-IDF values for that row
    tfidf_values = row[:-1]  # Assuming the last column is 'Protein_ID'

    # Extract the words and their TF-IDF values that have non-zero TF-IDF values
    non_zero_indices = tfidf_values[tfidf_values != 0].index
    non_zero_words = [(feature_names[i], tfidf_values[i]) for i in non_zero_indices]

    # Sort the words in descending order of their TF-IDF values
    sorted_words = sorted(non_zero_words, key=lambda x: x[1], reverse=True)

    return sorted_words

# Example usage:
protein_id = 'Q5K4F8'
sorted_words = get_non_zero_words(protein_id, df_train, vectorizer)

## Test_IDS

In [27]:
test_ids = np.load('/kaggle/input/test-ids-sorted-zong/t5_labels_sorted.npy')
test_ids_df =  pd.DataFrame(test_ids, columns = ['ID'])

In [28]:
test_ids_df

,ID
0,A0A023PXA5
1,A0A023PXB0
2,A0A023PXB5
3,A0A023PXB9
4,A0A023PXC2
...,...
141859,X5JB51
141860,X5M5N0
141861,X5M8U1
141862,X6R8D5


In [29]:
embed_ids_test = pd.read_csv('/kaggle/input/abstract-embeds-final/protein_ids_test_sparse.csv')

In [30]:
from scipy.sparse import load_npz
sparse_matrix_test = load_npz('/kaggle/input/abstract-embeds-final/tfidf_go_sparse_matrix_test.npz')

df_npz = pd.DataFrame.sparse.from_spmatrix(sparse_matrix_test)
df_npz['Protein_ID'] = embed_ids_test['Protein ID'].values

# Convert series_test_protein_ids to DataFrame
df_ids = test_ids_df

# Reset the index of df_npz to allow the merge
#df_npz_reset = df_npz.reset_index()

# Perform the merge with a left join
df_test = pd.merge(df_ids, df_npz, how='left', left_on='ID', right_on = 'Protein_ID')

# Fill NaN values with zeros
df_test[df_npz.columns] = df_test[df_npz.columns].fillna(0)

#Index 631 and 632 are duplicates. So drop one. Both are protein A0A1D6E0S8 and have the same embedding vector as well.
df_test = df_test.drop(632)

# Set 'ID' as the index of the DataFrame again
df_test.set_index('Protein_ID', inplace=True)
df_test = df_test.drop('ID', axis=1)


In [31]:
df_test

,0,1,2,3,4,5,6,7,8,9,...,10269,10270,10271,10272,10273,10274,10275,10276,10277,10278
Protein_ID,,,,,,,,,,,,,,,,,,,,,
A0A023PXA5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0A023PXB0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0A023PXB5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0A023PXB9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0A023PXC2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X5JB51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
X5M5N0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.126220,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
X5M8U1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025795,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
df_test

,0,1,2,3,4,5,6,7,8,9,...,10269,10270,10271,10272,10273,10274,10275,10276,10277,10278
Protein_ID,,,,,,,,,,,,,,,,,,,,,
A0A023PXA5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0A023PXB0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0A023PXB5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0A023PXB9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0A023PXC2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X5JB51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
X5M5N0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.126220,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
X5M8U1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025795,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
sorted_words = get_non_zero_words('Q60590', df_test, vectorizer)

In [34]:
sorted_words

[('alkenylglycerophosphoethanolamine', 0.31466981225574775),
 ('nicotianamine', 0.30988954042391853),
 ('stacked', 0.2937999993054977),
 ('xxvi', 0.25393019384891563),
 ('evoked', 0.22070849244018712),
 ('nucleated', 0.18991829247910175),
 ('phosphoribosylanthranilate', 0.1719405906056613),
 ('bent', 0.15450488111142333),
 ('joining', 0.15363811765798605),
 ('cycloisomerase', 0.14968307658766994),
 ('vasculogenesis', 0.14674879542159344),
 ('lengthening', 0.1218205422456459),
 ('picln', 0.12165340330148766),
 ('formate', 0.11769408608063059),
 ('acf', 0.11144567616441034),
 ('domains', 0.10988155311775148),
 ('midpiece', 0.10747054144388199),
 ('ionizing', 0.1050746593971932),
 ('dnag', 0.10186589134792698),
 ('mhf', 0.10078420136952583),
 ('isopentenyladenosine', 0.09961797132347255),
 ('anisomycin', 0.09497325592143471),
 ('chelatase', 0.09361019575202799),
 ('methylthiopentyl', 0.08876680220650868),
 ('activation', 0.08686819338280706),
 ('oliose', 0.08274176806896365),
 ('diacylgly

In [35]:
np.save('/kaggle/working/abstract_embeds_train_sorted_all.npy', df_train.values)

In [36]:
#np.save('/kaggle/working/abstract_embeds_test_sorted_all.npy', df_test.values)